### Classes Game

Vous trouverez ici les dernières versions des classes de base du jeu.



In [1]:
class Account :
    def __init__(self, cash=None):
        
        if (cash is None):
            self.cash = 100.0
        else:
            self.cash = cash

        self.cashInit = self.cash
        
        self.security = 0

    def buy(self, numberOfSecurity, unityPrice):
        
        cost = unityPrice * numberOfSecurity 
        
        if(self.cash - cost >= 0):
            self.cash = self.cash - cost
            self.security = self.security + numberOfSecurity
        else:
            raise NameError("Buy error : not enought money")

    def buyMax(self, unityPrice):

        numberOfSecurity = int(self.cash / unityPrice)

        self.buy(numberOfSecurity, unityPrice)

    def sell(self, numberOfSecurity, unityPrice):

        if(self.security - numberOfSecurity >= 0):
            self.security = self.security - numberOfSecurity
            self.cash = self.cash + numberOfSecurity * unityPrice
        else:
            raise NameError("Sell error : not enought coins")

    def sellMax(self, unityPrice):
        self.sell(self.security, unityPrice)

    def isOver(self):
        return self.security == 0 and self.cash == 0.0

    def getTheoricalValue(self, unityPrice):
        return (self.cash + self.security * unityPrice)
    
    def getTheoricalGain(self, unityPrice):
        return self.getTheoricalValue(unityPrice) - self.cashInit

    ''' 
    Méthode permettant d'afficher vos objets Account de manière conviviale. Cette méthode est
    automatiquement appelée quand un objet Account est donné en argument de la fonction 'print'
    '''
    def __str__(self):
        return "Cash: "+str(self.cash)+" & Security: "+str(self.security)+" & Cash gains: "+str(self.cash-self.cashInit)

### Game simple

Classe pour un jeu réalisant une prédiction seulement en fonction du prix journalier.

La série en paramètre est le prix journalier

Exemple d'utilisation:

    # création Account et Strategy
    account = Account(100)
    strategy = StrategyOracle()

    # création du jeu
    game = Game(serie.Close)
    game.setAccount(account)
    game.setStrategy(strategy)
    game.run()

In [2]:
class Game :

    '''
    Définition des variables appartenant à la classe, et non pas aux objets
    '''
    BUY = 0
    SELL = 1
    STILL = 2

    ACTIONS = ["BUY", "SELL", "STILL"]

    def __init__(self, serie, account=None, strategy=None):
        self.dates = serie.index
        self.prices = serie.to_numpy()

        self.account = account
        self.strategy = strategy
        self.clock = 0

    def setStrategy(self, strategy):
        self.strategy = strategy

    def setAccount(self, account):
        self.account = account
        self.clock = 0

    def run(self, debug=False):

        if(self.account is None):
            raise NameError("Account is not set")

        if(self.strategy is None):
            raise NameError("Strategy is not set")

        for unityPrice in self.prices:

            (action, amountOfSecurity) = self.strategy.getAction(self, unityPrice, self.dates[self.clock], debug)

            actionStr = Game.ACTIONS[action]

            if debug:
                print("--- ", actionStr, " ",amountOfSecurity, "securities at clock ", self.clock)

            if(self.account.isOver()):
                break;

            try:
                if(action == Game.BUY):
                    self.account.buy(amountOfSecurity, unityPrice)
                elif(action == Game.SELL):
                    self.account.sell(amountOfSecurity, unityPrice)
                else:
                    pass
            except NameError as error:
                if debug:
                    print(error)

            if debug:
                print(self.account)

            self.clock = self.clock+1

    def getTheoricalValue(self):
        return self.account.getTheoricalValue(self.prices[-1])

    def getTheoricalGain(self):
        return self.account.getTheoricalGain(self.prices[-1])

    def __str__(self):
        return "Game after "+str(self.clock)+"/"+str(len(self.prices))+" steps - "+str(self.account)


### Game complex

Classe pour un jeu réalisant une prédiction en fonction de données complexes (une matrice de données).

La construction de l'objet nécessite de prendre un datframe, le nom de la colonne avec le prix journalier et la liste des colonnes à utiliser pour réaliser la prédiction.

Exemple d'utilisation:

    # création Account et Strategy
    account = Account(100)
    strategy = StrategyOracle()    
    
    # création du jeu
    game = GameComplex(serie, columnsPrice=["Close"], columnsPrediction=["Close"])
    game.setAccount(account)
    game.setStrategy(strategy)

    game.run()


In [3]:
class GameComplex :

    '''
    Définition des variables appartenant à la classe, et non pas aux objets
    '''
    BUY = 0
    SELL = 1
    STILL = 2

    ACTIONS = ["BUY", "SELL", "STILL"]

    def __init__(self, data, columnsPrice="Close", columnsPrediction=["Close"], account=None, strategy=None):
        self.dates = data.index
        self.prices = data[columnsPrice].to_numpy()
        self.seriePrediction = data[columnsPrediction].to_numpy()
           
        self.account = account
        self.strategy = strategy
        self.clock = 0
        
    def setStrategy(self, strategy):
        self.strategy = strategy

    def setAccount(self, account):
        self.account = account

    def run(self, debug=False):
        
        if(self.account is None):
            raise NameError("Account is not set")
        
        if(self.strategy is None):
            raise NameError("Strategy is not set")
        
        self.clock = 0

        for _ in range(0, len(self.prices)):

            unityPrice = self.prices[self.clock]
            dataPrediction = self.seriePrediction[self.clock, :]
            
            if debug:
                print("dataPrediction:", dataPrediction, " & price:", unityPrice)
            
            (action, amountOfSecurity) = self.strategy.getAction(self, dataPrediction, self.dates[self.clock], debug)

            actionStr = Game.ACTIONS[action]

            if debug:
                print("--- ", actionStr, " ",amountOfSecurity, "securities at clock ", self.clock)

            if(self.account.isOver()):
                break;

            try:
                if(action == Game.BUY):
                    self.account.buy(amountOfSecurity, unityPrice)
                elif(action == Game.SELL):
                    self.account.sell(amountOfSecurity, unityPrice)
                else:
                    pass
            except NameError as error:
                if debug:
                    print(error)

            if debug:
                print(self.account)
                
            self.clock = self.clock+1

    def getTheoricalValue(self):
        return self.account.getTheoricalValue(self.prices[-1])
    
    def getTheoricalGain(self):
        return self.account.getTheoricalGain(self.prices[-1])

    def __str__(self):
        return "Game after "+str(self.clock)+"/"+str(len(self.prices))+" steps - "+str(self.account)

### Classe IStrategy

In [4]:
class IStrategy :

    def __init__(self, name="IStrategy"):
        self.name = name

    def isRandom(self):
        return False

    def getAction(self, game, unityPrice, date, debug=False):
        '''
        @return (action, amountOfSecurity)
        '''
        pass

    def getName(self):
        '''
        @return the name of the strategy
        '''
        return self.name

### Classe EvalGame

In [5]:
class EvalGame:

    def __init__(self, initialAmount=100):
        self.game = None
        self.initialAmount = initialAmount
        self.results = []
    
    def setGame(self, serie, strategy):
        self.game = Game(serie)
        self.game.setAccount(Account(self.initialAmount))
        self.game.setStrategy(strategy)

    def eval(self, serie, strategy, iteration=10, debug=False):
        ''' 
        eval serie on strategy

        @params serie: serie to assess
        @params strategy : strategy to assess
        @params iteration: number of iteration for random strategy

        @return result: gain for the strategy on the whole serie 
        
        '''        
        self.results = []

        realIteration =1
        if(strategy.isRandom()):
            realIteration = iteration 

        for i in range(0, realIteration):
            self.setGame(serie, strategy)
            
            if(debug):
                print("Game:", self.game)
            
            self.game.run(debug=debug)

            score = self.game.getTheoricalGain()
            self.results.append(score)

            if debug:
                print("score[", i, "]: ", score) 

        return self.results   

    def evalTime(self, serie, strategy, iteration=10, debug=False):
        ''' 
        eval serie on strategy for each date of the serie

        @params serie: serie to assess
        @params strategy : strategy to assess
        @params iteration: number of iteration for strategy

        @return results: list of gains, one for each start date 
        
        '''     
        results = []

        for t in serie.index: # range(0, serie.shape[0]-1):
            serieT = serie.loc[t:]

            if debug:
                print("serieT:", serieT.shape, " first date:", serieT.index[-1])

            result = self.eval(serieT, strategy, iteration=iteration, debug=False)
            results.append(np.mean(result))

        return pd.Series(data=results, index=serie.index)

    def evalSeries(self, series, strategy, iteration=10, debug=False):
        '''
        Eval a list of series on strategy. Series are not supposed to have the same length.

        @params series: list of series
        @params strategy : strategy to test on the series

        @return list of series, obtained with evalTime()
        '''
        results = []

        for serie in series:
            results.append(self.evalTime(serie, strategy, iteration, debug))
        
        return results
    
    def evalStrategies(self, serie, strategies, iteration=10, debug=False):
        '''
        Eval a list of series on . Series are not supposed to have the same length.

        @params serie: serie
        @params strategies: strategies to test on the serie

        @return a DataFrame, with one column per Strategy
        '''

        results = pd.DataFrame()

        for strategy in strategies:
            result = self.evalTime(serie, strategy, iteration, debug)
            
            results[strategy.getName()] = result

        return results

    def getResults(self):
        return self.results